In [ ]:
import torch
import torch.nn as nn
import statistics
import torchvision.models as models ## used to load the pytorch models for vision


In [ ]:
class CNNtoRNN(nn.Module): # cnn to rnn is hooked here.
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size)
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions)
        return outputs

    def caption_image(self, image, vocabulary, max_length=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoderCNN(image).unsqueeze(0)
            states = None

            for _ in range(max_length): ## upto the max_length words prediction, 50 here.
                hiddens, states = self.decoderRNN.lstm(x, states)  ## at beginning it will be initialized as 0.
                output = self.decoderRNN.linear(hiddens.squeeze(0))
                predicted = output.argmax(1) # so we taking word with higgest probablity.
                result_caption.append(predicted.item())
                x = self.decoderRNN.embed(predicted).unsqueeze(0) # taking the predicted word.

                if vocabulary.itos[predicted.item()] == "<EOS>": # check if vocab is equal to end of sentence then break.
                    break

        return [vocabulary.itos[idx] for idx in result_caption]